**Install Required Packages**

In [14]:
!pip install google-generativeai tavily-python

**Import Libraries**

In [15]:
import os
import time
from typing import List
from dataclasses import dataclass
import google.generativeai as genai
from tavily import TavilyClient
import getpass

**Data Classes**

In [16]:
@dataclass
class SearchResult:
    title: str
    content: str
    url: str

@dataclass
class ResearchQuestion:
    question: str
    search_results: List[SearchResult]
    summary: str = ""

**ReAct Agent Class**

In [35]:
class ReActAgent:
    def __init__(self, gemini_api_key, tavily_api_key):
        self.research_questions: List[ResearchQuestion] = []

        # Initialize Gemini with correct model name
        genai.configure(api_key=gemini_api_key)
        self.llm_client = genai.GenerativeModel("gemini-1.5-flash")

        # Initialize Tavily
        self.search_client = TavilyClient(api_key=tavily_api_key)
        print("✅ ReAct Agent initialized successfully")

    def _call_gemini(self, prompt: str) -> str:
        try:
            response = self.llm_client.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.7,
                    max_output_tokens=1000,
                )
            )
            return response.text.strip() if response.text else ""
        except Exception as e:
            print(f"Error calling Gemini: {e}")
            # Try alternative model if flash fails
            try:
                alt_client = genai.GenerativeModel("gemini-3.5-pro")
                response = alt_client.generate_content(
                    prompt,
                    generation_config=genai.types.GenerationConfig(
                        temperature=0.7,
                        max_output_tokens=1000,
                    )
                )
                return response.text.strip() if response.text else ""
            except Exception as e2:
                print(f"Error with alternative model: {e2}")
                return ""

    def generate_research_questions(self, topic: str, num_questions: int = 5) -> List[str]:
        print(f"🧠 REASONING: Generating research questions for: '{topic}'")

        prompt = f"""Generate {num_questions} specific research questions about "{topic}".
        Requirements:
        - Each question should be clear and searchable
        - Cover different aspects of the topic
        - Format as numbered list (1. 2. 3. etc.)

        Topic: {topic}

        Example format:
        1. What are the main causes of [topic]?
        2. How does [topic] affect [specific area]?
        3. What are the current solutions for [topic]?"""

        response = self._call_gemini(prompt)

        questions = []
        for line in response.split('\n'):
            line = line.strip()
            if line and line[0].isdigit():
                question = line.split('.', 1)[-1].strip()
                if question:
                    questions.append(question)

        print(f"✅ Generated {len(questions)} questions")
        return questions[:num_questions]

    def search_web(self, query: str) -> List[SearchResult]:
        print(f"🔍 ACTING: Searching for: '{query}'")

        try:
            search_response = self.search_client.search(query=query, max_results=3)

            results = []
            for result in search_response.get('results', []):
                search_result = SearchResult(
                    title=result.get('title', ''),
                    content=result.get('content', ''),
                    url=result.get('url', '')
                )
                results.append(search_result)

            print(f"✅ Found {len(results)} results")
            return results
        except Exception as e:
            print(f"Error in search: {e}")
            return []

    def summarize_findings(self, question: str, search_results: List[SearchResult]) -> str:
        if not search_results:
            return "No information found."

        print("🧠 REASONING: Summarizing findings...")

        context = ""
        for i, result in enumerate(search_results, 1):
            context += f"Source {i}: {result.title}\n{result.content[:500]}...\n\n"

        prompt = f"""Answer this question based on the search results provided:

Question: {question}

Search Results:
{context}

Instructions:
- Provide a clear, comprehensive answer in 2-3 paragraphs
- Use information from the sources provided
- Be factual and objective
- If sources conflict, mention the different perspectives

Answer:"""

        summary = self._call_gemini(prompt)
        return summary if summary else "Unable to generate summary."

    def research_topic(self, topic: str, num_questions: int = 5):
        print(f"🚀 Starting research on: '{topic}'")
        print("=" * 50)

        # Step 1: Generate questions
        questions = self.generate_research_questions(topic, num_questions)

        if not questions:
            print("❌ Failed to generate questions. Please check your API key and internet connection.")
            return

        # Step 2: Research each question
        for i, question in enumerate(questions, 1):
            print(f"\n📋 Question {i}: {question}")

            # Search web
            search_results = self.search_web(question)

            # Summarize findings
            summary = self.summarize_findings(question, search_results)

            # Store results
            research_q = ResearchQuestion(
                question=question,
                search_results=search_results,
                summary=summary
            )
            self.research_questions.append(research_q)

            print(f"📝 Summary: {summary[:100]}...")
            time.sleep(1)  # Respectful delay

        print("\n✅ Research completed!")

    def generate_report(self, topic: str) -> str:
        if not self.research_questions:
            return "No research data available."

        report = f"""# Research Report: {topic}

## Introduction
This report presents research findings on "{topic}" using the ReAct (Reasoning + Acting) pattern for web research.

## Research Questions and Findings
"""

        for i, rq in enumerate(self.research_questions, 1):
            report += f"\n### {i}. {rq.question}\n\n{rq.summary}\n\n"

            if rq.search_results:
                report += "**Sources:**\n"
                for j, result in enumerate(rq.search_results, 1):
                    report += f"{j}. [{result.title}]({result.url})\n"
            report += "\n"

        report += f"\n## Conclusion\n"
        report += f"This research covered {len(self.research_questions)} key aspects of {topic}, "
        report += f"providing comprehensive insights through web search and analysis.\n"

        return report

**Setup API Keys**

In [42]:
print("Enter your API keys:")
GEMINI_API_KEY = getpass.getpass("Gemini API Key: ")
TAVILY_API_KEY = getpass.getpass("Tavily API Key: ")


try:
    agent = ReActAgent(GEMINI_API_KEY, TAVILY_API_KEY)
except Exception as e:
    print(f"Error initializing agent: {e}")
    print("Please check your API keys and try again.")

Enter your API keys:
Gemini API Key: ··········
Tavily API Key: ··········
✅ ReAct Agent initialized successfully


** Run Research**

In [44]:
research_topic = input("🔎 Enter the topic you want to research: ")

try:
    agent.research_topic(research_topic, num_questions=5)
except Exception as e:
    print(f"Error during research: {e}")

🔎 Enter the topic you want to research: SNS Square Consultancy Services Pvt Ltd
🚀 Starting research on: 'SNS Square Consultancy Services Pvt Ltd'
🧠 REASONING: Generating research questions for: 'SNS Square Consultancy Services Pvt Ltd'
✅ Generated 5 questions

📋 Question 1: What types of consultancy services does SNS Square Consultancy Services Pvt Ltd primarily offer, and what are their key industry specializations?
🔍 ACTING: Searching for: 'What types of consultancy services does SNS Square Consultancy Services Pvt Ltd primarily offer, and what are their key industry specializations?'
✅ Found 3 results
🧠 REASONING: Summarizing findings...
📝 Summary: SNS Square Consultancy Services Pvt Ltd primarily offers IT consultancy services, focusing on helpin...

📋 Question 2: What is the client base and geographic reach of SNS Square Consultancy Services Pvt Ltd, based on publicly available information and news reports?
🔍 ACTING: Searching for: 'What is the client base and geographic reach of 

Install Required Packages

**Generate and Display Report**

**Final Report**

In [40]:
report = agent.generate_report(research_topic)
print("\n" + "="*60)
print("FINAL REPORT")
print("="*60)
print(report)


FINAL REPORT
# Research Report: SNS Square Consultancy Services Pvt Ltd

## Introduction
This report presents research findings on "SNS Square Consultancy Services Pvt Ltd" using the ReAct (Reasoning + Acting) pattern for web research.

## Research Questions and Findings

### 1. What is the correlation between increased atmospheric CO2 concentration and the frequency of extreme weather events (hurricanes, droughts, heatwaves) in the Mediterranean region since 1980?

Source 1 is irrelevant as it provides a current weather report for Santiago, Chile, and not data on the Mediterranean region's climate trends since 1980.  Source 2 indicates a direct correlation between increased CO2 concentrations and an increase in extreme heat events, even without a change in average global temperatures.  While this study doesn't specifically focus on the Mediterranean, it supports the general principle that higher CO2 levels contribute to more frequent extreme weather.

Source 3, from the IPCC AR6 repo

**Save Report**

In [41]:
with open(f"research_report_{research_topic.replace(' ', '_')}.md", 'w') as f:
    f.write(report)
print(f"Report saved as: research_report_{research_topic.replace(' ', '_')}.md")

Report saved as: research_report_SNS_Square_Consultancy_Services_Pvt_Ltd.md
